```
@beelzebruno
2022
```

In [4]:
import folium
from folium import plugins
import pandas as pd

<hr />

# The Beer Route

<hr /><br />

Ok, its been five weeks now from the time I have started the whole process, by the time I am writing these lines, of course, anyway, since then I've been reviwing past contents from my bachelors which are also the foundations of algorithms and computer science, I am opening different notebooks with different implemented code to exercise and practice the algorithms || yes I know I could write once and import from a `src` module ||, but thats not the point of this repository. Here I am repeating same data structures in different time and self humor coding to exercice myself, my skill and my knowledge, through different kind of problems, some from code challenges from ther websites, some from books and some that i have created myself (see Highlands in this [notebook here](https://github.com/brunolcarli/GooGraph/blob/master/graph_search.ipynb)).

<br />

Ok, lets stop talking, I feel like i got some improvement since highlands, so lets have another try on graphs. I ~~love~~like having a beer during happy hours with my friends, and of course there are some pubs that I am more likely to drink at. Suppose it is my birthday and a I have selected eight from my favorite pubs and we have to drink one beer at each one without repeating locations, of course we should start on our most favorite and classic ever.

<br />

The favorite places are ploted on the map:

In [5]:
pub_geolocations = {
    'Jokers': (-25.4280205, -49.2723036),
    'Villa Bambu': (-25.4267986, -49.2758894),
    'Slainte': (-25.435735, -49.2873796),
    'Sheridans': (-25.4454165, -49.2969174),
    'Txapela': (-25.446419,-49.2983345),
    'Tatas': (-25.4374191, -49.2766666),
    'James': (-25.4334848, -49.2838167),
    'Blood Rock': (-25.424682, -49.2789355),
}


cwb_map = folium.Map(location=pub_geolocations['Villa Bambu'], zoom_start=14)
for k, v in pub_geolocations.items():
    folium.Marker(v, popup=k).add_to(cwb_map)

cwb_map

Out:

![cwb_beer_route](https://i.ibb.co/Y3BZtK6/cwb-beer-route.png)

Now instead of following booksand guides and pseudocode approaches, im gonna try to implement an approach by myself based on the other content a have followed until this moment.

Now lets think about simple nodes, which have their names and possibilities with their distances, just like highlands.

<br />

A Graph is a set of nodes built over a heuristic hashtable.

<br />

To walk through the nodes, I will implement a new data structure called `Cursor`, much like Russel & Norvig going to call an **Agent** maybe. I will use search algorithms search paths (edges) between the nodes, following some rules (like number of steps or even **cant repeat the same place twice** as stated on the abstraction). So the Cursor is the one who retain search and movement on the graph.

In [9]:
# Heuristic of the mean min walking distance in meters by google maps
pub_distances = {
    'Jokers': {
        'Jokers': 0,
        'Villa Bambu': 450,
        'Slainte': 2000,
        'Sheridans': 3500,
        'Txapela': 3700,
        'Tatas': 1500,
        'James': 1500,
        'Blood Rock': 900
    },
    'Villa Bambu': {
        'Jokers': 450,
        'Villa Bambu': 0,
        'Slainte': 1900,
        'Sheridans': 3500,
        'Txapela': 3600,
        'Tatas': 1500,
        'James': 1400,
        'Blood Rock': 450
    },
    'Slainte': {
        'Jokers': 2000,
        'Villa Bambu': 1900,
        'Slainte': 0,
        'Sheridans': 2000,
        'Txapela': 2100,
        'Tatas': 1300,
        'James': 500,
        'Blood Rock': 2000
    },
    'Sheridans': {
        'Jokers': 3500,
        'Villa Bambu': 3500,
        'Slainte': 2000,
        'Sheridans': 0,
        'Txapela': 170,
        'Tatas': 2300,
        'James': 2300,
        'Blood Rock': 3500
    },
    'Txapela': {
        'Jokers': 3700,
        'Villa Bambu': 3600,
        'Slainte': 2100,
        'Sheridans': 170,
        'Txapela': 0,
        'Tatas': 2500,
        'James': 2500,
        'Blood Rock': 3700
    },
    'Tatas': {
        'Jokers': 1500,
        'Villa Bambu': 1400,
        'Slainte': 1300,
        'Sheridans': 2300,
        'Txapela': 2500,
        'Tatas': 0,
        'James': 1100,
        'Blood Rock': 1700
    },
    'James': {
        'Jokers': 1500,
        'Villa Bambu': 1400,
        'Slainte': 1300,
        'Sheridans': 2300,
        'Txapela': 2500,
        'Tatas': 1100,
        'James': 0,
        'Blood Rock': 1500
    },
    'Blood Rock': {
        'Jokers': 900,
        'Villa Bambu': 450,
        'Slainte': 200,
        'Sheridans': 3500,
        'Txapela': 3700,
        'Tatas': 1700,
        'James': 1500,
        'Blood Rock': 0
    }
}

In [10]:
class Node:
    """ Node is a static point in a network mesh (Graph) """
    def __init__(self, name, heuristic):
        self.name = name
        self.heuristic = heuristic

    def __repr__(self):
        return f'{self.name}'
        
    def distance_to(self, node):
        """ Returns the distance value to argument Node object """
        return self.heuristic[self.name][node.name]

    def get_possibilities(self):
        """ Return a list of Node connection (edges) from this Node """
        return [Node(name, self.heuristic) for name in self.heuristic[self.name]]


class Cursor:
    """
    The cursor will move node to node of a graph.
    The cursor moves by searching a (good or not) option of movement.
    A Rule defined by the programmer says the cursor visit a node more tha once.
    The cursor store the total distance and the node count walked.
    """
    def __init__(self, graph, root):
        self.root = root
        self.current = root
        self.last_position = root
        self.visited = [root.name]
        self.total_distance = 0
        self.movements = 0
        # TODO: Implement computational effort spent

    def __repr__(self):
        return f'Cursor at {self.current} from {self.last_position}'
        
    def move(self, node):
        """ Move cursor to argument node """
        self.total_distance += self.current.distance_to(node)
        self.last_position = self.current
        self.current = node
        self.visited.append(node.name)
        self.movements += 1

        return self

    def search_best_option(self, strategy='dummy'):
        """
        This is the very soul of the cursor, which represents
        his very own existance. This method should select the
        next position (node) the cursor will move into the graph.
        """
        options = [option for option in self.current.get_possibilities()
                   if option.name not in self.visited]

        if not options:
            return None

        # scratch sort the list
        # TODO: implelemnt more sort options
        # TODO: Choose sort method from parameter
        length = len(options)
        for i in range(1, length):
            j = i
            right = self.current.distance_to(options[j])
            left = self.current.distance_to(options[j-1])
            while (right < left) and (j != 0):
                options[j], options[j-1] = options[j-1], options[j]
                j -= 1

        # dummy returns the first option
        # TODO: implement other search methods
        # TODO: Choose search method by parameter
        return options[0]


class Graph:
    """
        A graph is a set of nodes based.
        This graph is built over a primary known heuristic hashmap.
    """
    def __init__(self, heuristic):
        self.nodes = {name: Node(name, heuristic) for name in heuristic}

    def __repr__(self):
        return ' | '.join(self.nodes)

In [11]:
# Driver code
START = 'Villa Bambu'
pub_graph = Graph(pub_distances)
pub_cursor = Cursor(pub_graph, pub_graph.nodes[START])

records = [[str(pub_cursor), pub_cursor.total_distance, pub_cursor.movements]]
while True:
    option = pub_cursor.search_best_option()
    if not option:
        break
    pub_cursor.move(option)
    records.append([str(pub_cursor), pub_cursor.total_distance, pub_cursor.movements])

pub_df = pd.DataFrame(records, columns=['Cursor', 'Distance', 'Movements'])
pub_df

,Cursor,Distance,Movements
0,Cursor at Villa Bambu from Villa Bambu,0,0
1,Cursor at Blood Rock from Villa Bambu,450,1
2,Cursor at James from Blood Rock,1950,2
3,Cursor at Tatas from James,3050,3
4,Cursor at Slainte from Tatas,4350,4
5,Cursor at Jokers from Slainte,6350,5
6,Cursor at Sheridans from Jokers,9850,6
7,Cursor at Txapela from Sheridans,10020,7
